<a href="https://colab.research.google.com/github/ArthurChan-1111/Bitcoin_prediction/blob/main/Master_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!git clone https://github.com/ArthurChan-1111/Bitcoin_prediction.git

%cd Bitcoin_prediction

import pandas as pd
import numpy as np

df_bitcoin_by_day = pd.read_csv('btc_1d_data_2018_to_2025.csv', on_bad_lines='skip', lineterminator='\n')


Cloning into 'Bitcoin_prediction'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 166.79 KiB | 2.03 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction


In [17]:
df_bitcoin_by_day.describe().T

,count,mean,std,min,25%,50%,75%,max
Open,2638.0,3.007471e+04,2.462686e+04,3.211710e+03,9.141495e+03,2.316573e+04,4.456562e+04,1.061438e+05
High,2638.0,3.076510e+04,2.511586e+04,3.276500e+03,9.304870e+03,2.373610e+04,4.616359e+04,1.083530e+05
Low,2638.0,2.933413e+04,2.411405e+04,3.156260e+03,8.874750e+03,2.270910e+04,4.336132e+04,1.053215e+05
Close,2638.0,3.010409e+04,2.465340e+04,3.211720e+03,9.141368e+03,2.317984e+04,4.466510e+04,1.061438e+05
Volume,2638.0,6.986922e+04,8.017029e+04,4.811771e+02,2.935439e+04,4.436260e+04,7.258744e+04,7.607054e+05
Quote asset volume,2638.0,1.806400e+09,2.007644e+09,1.177017e+07,3.825199e+08,1.126328e+09,2.526423e+09,1.746531e+10
Number of trades,2638.0,1.740156e+06,2.117254e+06,1.241700e+04,4.140775e+05,1.018162e+06,1.931050e+06,1.522359e+07
Taker buy base asset volume,2638.0,3.472933e+04,3.989757e+04,2.313948e+02,1.478038e+04,2.222689e+04,3.584608e+04,3.747756e+05
Taker buy quote asset volume,2638.0,8.943653e+08,9.983054e+08,6.532639e+06,1.907035e+08,5.474179e+08,1.260436e+09,8.783916e+09
Ignore,2638.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [18]:
def calculate_indicators(df):
    # Calculate RSI
    def rsi(series, period=14):
        delta = series.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    # Calculate CCI
    def cci(series, period=14):
        typical_price = (series['High'] + series['Low'] + series['Close']) / 3
        ma = typical_price.rolling(window=period).mean()
        mad = (typical_price - ma).abs().rolling(window=period).mean()
        return (typical_price - ma) / (0.015 * mad)

    # Calculate Bollinger Bands
    def bollinger_bands(series, window=20, num_std_dev=2):
        sma = series['Close'].rolling(window=window).mean()
        rstd = series['Close'].rolling(window=window).std()
        upper_band = sma + (rstd * num_std_dev)
        lower_band = sma - (rstd * num_std_dev)
        return upper_band, lower_band

    # Calculating indicators
    df['RSI7'] = rsi(df['Close'], period=7)
    df['RSI14'] = rsi(df['Close'], period=14)
    df['CCI7'] = cci(df, period=7)
    df['CCI14'] = cci(df, period=14)
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['Bollinger_Upper'], df['Bollinger_Lower'] = bollinger_bands(df)

    #Cup and handle signal
    df['Cup_Handle_Signal'] = np.nan

    for i in range(3, len(df) - 3):  # Avoiding the edges of the DataFrame
        cup_bottom = df['Close'][i-3:i+1].min()  # Minimum price in the last 4 periods (cup)
        cup_top = df['Close'][i+1:i+4].max()  # Maximum price in the next 3 periods (handle)

        # Defining criteria for Cup and Handle
        if (df['Close'][i] == cup_bottom) and (cup_top > cup_bottom * 1.05):
            df['Cup_Handle_Signal'].iloc[i] = 1  # Signal for Cup and Handle
    #/Cup and handle signal

    # Set Binary Signal
    df['Signal_Up'] = 0  # Default to 0
    df['Signal_Up'].iloc[1:] = np.where(df['Close'].iloc[1:].values > df['Close'].iloc[:-1].values, 1, 0)

    return df

calculate_indicators(df_bitcoin_by_day)

df_bitcoin_by_day.tail()

<ipython-input-18-9489d2813c37>:42: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Cup_Handle_Signal'].iloc[i] = 1  # Signal for Cup and Handle
<ipython-input-18-9489d2813c37>:42: SettingWithCopyWarning: 
A value is trying to be set on a 

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,...,Ignore,RSI7,RSI14,CCI7,CCI14,MA50,Bollinger_Upper,Bollinger_Lower,Cup_Handle_Signal,Signal_Up
2633,2025-03-18,84010.02,84021.74,83017.88,83018.33,1110.53302,2025-03-18 23:59:59.999,9.266588e+07,179190,436.70291,...,0,68.363371,41.512263,3.070044,-29.091901,92459.5592,92228.296511,78262.968489,NaN,0
2634,2025-03-19,82715.03,83280.00,82603.20,83010.87,1360.52066,2025-03-19 23:59:59.999,1.129231e+08,209016,829.38438,...,0,46.272092,33.562588,-25.112016,-27.920901,92093.0662,92100.209554,78046.600446,NaN,0
2635,2025-03-20,86845.93,87453.67,85733.11,85905.91,3146.08729,2025-03-20 23:59:59.999,2.723224e+08,399959,1421.72638,...,0,61.212357,41.838930,151.593782,67.947207,91736.5196,92177.214055,78294.227945,NaN,1
2636,2025-03-21,84223.38,84748.44,84180.09,84711.21,656.96798,2025-03-21 23:59:59.999,5.548234e+07,117589,341.94482,...,0,69.261525,45.401429,29.070907,27.628368,91336.2850,92181.597884,78303.586116,NaN,0
2637,2025-03-22,84088.79,84338.98,83989.58,84037.82,534.27259,2025-03-22 23:59:59.999,4.496609e+07,86830,231.88751,...,0,48.410583,44.864757,4.511839,21.414159,90968.4502,90226.130479,79235.835521,NaN,0


In [19]:
df_bitcoin_by_day[df_bitcoin_by_day['Cup_Handle_Signal']==1]

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,...,Ignore,RSI7,RSI14,CCI7,CCI14,MA50,Bollinger_Upper,Bollinger_Lower,Cup_Handle_Signal,Signal_Up
10,2018-01-11,14940.00,14968.68,11400.00,13238.78,33554.723751,2018-01-11 23:59:59.999,4.465876e+08,239416,16422.019980,...,0,36.720327,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
15,2018-01-16,13500.00,13542.93,9035.00,10900.00,63401.169175,2018-01-16 23:59:59.999,7.221508e+08,418243,30353.401717,...,0,23.430502,33.799160,-136.779039,NaN,NaN,NaN,NaN,1.0,0
21,2018-01-22,11530.00,11926.35,9900.24,10760.05,43752.606791,2018-01-22 23:59:59.999,4.757165e+08,363264,21300.575828,...,0,29.046341,31.310687,-26.090353,NaN,NaN,17622.136939,9576.696061,1.0,0
35,2018-02-05,8179.99,8382.80,6625.00,6939.99,63403.182579,2018-02-05 23:59:59.999,4.711662e+08,533510,29857.358390,...,0,7.401508,23.945420,-91.822069,-158.554968,NaN,13315.272814,7734.331186,1.0,0
54,2018-02-24,10131.04,10496.97,9352.00,9694.51,40888.156299,2018-02-24 23:59:59.999,4.056186e+08,337918,18646.870961,...,0,31.422921,57.446809,-66.253706,-3.647601,10925.0052,11887.569418,6728.535582,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,2024-12-10,97276.48,98270.00,94256.54,96593.00,51708.689330,2024-12-10 23:59:59.999,4.988660e+09,11556189,24523.095860,...,0,52.845779,60.434438,-80.237935,-32.431052,85284.5266,101438.275018,92770.938982,1.0,0
2569,2025-01-13,94545.07,95940.00,89256.69,94536.10,42619.564230,2025-01-13 23:59:59.999,3.930666e+09,5740963,19473.880420,...,0,18.043222,54.057763,-45.537676,-85.450681,97358.1410,100732.248260,90914.943740,1.0,0
2576,2025-01-20,101331.57,101677.71,99550.00,100055.43,8487.795940,2025-01-20 23:59:59.999,8.526351e+08,1147358,4171.159850,...,0,67.757285,46.048666,-5.150435,64.698614,98148.7568,104795.057766,91066.108234,1.0,0
2615,2025-02-28,84708.57,84892.85,81111.00,82659.59,7040.171990,2025-02-28 23:59:59.999,5.823066e+08,718269,3035.236560,...,0,11.107304,24.191228,-125.709629,-245.808194,98014.6960,104040.591864,85415.648136,1.0,0


In [21]:
df_bitcoin_by_day[df_bitcoin_by_day['Signal_Up']==1]

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,...,Ignore,RSI7,RSI14,CCI7,CCI14,MA50,Bollinger_Upper,Bollinger_Lower,Cup_Handle_Signal,Signal_Up
1,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2018-01-02 23:59:59.999,2.797171e+08,177728,11346.326739,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2018-01-03 23:59:59.999,2.361169e+08,162787,8994.953566,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,2018-01-04 23:59:59.999,3.127816e+08,170310,12680.812951,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,2018-01-05 23:59:59.999,3.693220e+08,192969,13346.622293,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,2018-01-06,16960.39,17143.13,16011.21,17069.79,18571.457508,2018-01-06 23:59:59.999,3.092169e+08,158242,11007.164056,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2620,2025-03-05,87281.98,90933.82,86334.53,90283.57,36649.484150,2025-03-05 23:59:59.999,3.254229e+09,7716640,18359.331920,...,0,60.149200,44.278219,40.253500,-23.914560,97367.9326,103117.912039,81965.731961,NaN,1
2627,2025-03-12,82932.99,83591.07,82604.95,83488.72,1624.191460,2025-03-12 23:59:59.999,1.349775e+08,311866,977.529250,...,0,28.378680,49.162174,-32.037434,-39.030067,94897.7358,99531.739980,77298.730020,NaN,1
2628,2025-03-13,83680.12,84000.00,83312.92,83915.10,1854.128840,2025-03-13 23:59:59.999,1.552110e+08,273157,940.659960,...,0,30.945065,47.090678,-0.568450,-32.759954,94501.9046,97947.414677,77444.565323,NaN,1
2630,2025-03-15,83983.19,84583.04,83964.57,84227.92,782.819720,2025-03-15 23:59:59.999,6.596448e+07,131694,425.748270,...,0,43.619037,49.595132,40.758207,-29.557826,93644.6198,95043.867155,77656.577845,NaN,1


In [23]:
def calculate_cup_probability(df):
    # Ensure 'Cup_Handle_Signal' and 'Signal_Up' columns are available
    if 'Cup_Handle_Signal' not in df.columns or 'Signal_Up' not in df.columns:
        raise ValueError("DataFrame must contain 'Cup_Handle_Signal' and 'Signal_Up' columns.")

    # Identify the indices of Cup and Handle signals
    cup_handle_indices = df.index[df['Cup_Handle_Signal'] == 1].tolist()

    # Count occurrences of 'Up' signals the day after Cup and Handle signals
    up_next_day_count = 0
    total_cup_handle_count = len(cup_handle_indices)

    for index in cup_handle_indices:
        if index + 1 < len(df):  # Ensure we don't go out of bounds
            if df['Signal_Up'].iloc[index + 1] == 1:  # Check for 'Up' signal
                up_next_day_count += 1

    # Calculate probability
    probability = up_next_day_count / total_cup_handle_count if total_cup_handle_count > 0 else 0

    return probability

calculate_cup_probability(df_bitcoin_by_day)

0.8397435897435898